In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip install glove-python-binary

     |████████████████████████████████| 948 kB 5.0 MB/s 


In [ ]:
from glove import Glove
glove=Glove()
glove_model=glove.load("/content/gdrive/MyDrive/FYP/CAA_NRC/Glove/glove.model")

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/gdrive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=pd.get_dummies(df["label"])


In [ ]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [ ]:

import tensorflow as tf
maxlen=maxi
max_words=len(glove_model.dictionary)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

In [ ]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in glove_model.dictionary:
    embed_vector=glove_model.word_vectors[glove_model.dictionary[word]]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [ ]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.10990729,  0.04230337,  0.05654864, ..., -0.20832376,
        -0.03223338, -0.21452594],
       [ 0.08457289, -0.06926741, -0.00822759, ..., -0.15099157,
        -0.02485671, -0.19643595],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,LSTM,Dense
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=maxi,embeddings_initializer=Constant(embed_matrix)))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=False)) 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(3,activation='softmax'))

In [ ]:
import time
start = time.time()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/10
37/37 [==============================] - 12s 55ms/step - loss: 0.1811 - accuracy: 0.9325
Epoch 2/10
37/37 [==============================] - 2s 51ms/step - loss: 0.1710 - accuracy: 0.9325
Epoch 3/10
37/37 [==============================] - 2s 47ms/step - loss: 0.1420 - accuracy: 0.9402
Epoch 4/10
37/37 [==============================] - 2s 47ms/step - loss: 0.1447 - accuracy: 0.9501
Epoch 5/10
37/37 [==============================] - 2s 51ms/step - loss: 0.1206 - accuracy: 0.9535
Epoch 6/10
37/37 [==============================] - 2s 52ms/step - loss: 0.1188 - accuracy: 0.9510
Epoch 7/10
37/37 [==============================] - 2s 48ms/step - loss: 0.1173 - accuracy: 0.9480
Epoch 8/10
37/37 [==============================] - 2s 47ms/step - loss: 0.1105 - accuracy: 0.9527
Epoch 9/10
37/37 [==============================] - 2s 49ms/step - loss: 0.1162 - accuracy: 0.9574
Epoch 10/10
37/37 [==============================] - 2s 45ms/step - loss: 0.1134 - accuracy: 0.9484
Training

In [ ]:
# y_pred=model.predict(X_test)
# for i in range(len(y_pred)):
#   if y_pred[i]>0.5:
#     y_pred[i]=1
#   else:
#     y_pred[i]=0

In [ ]:
import numpy as np
y_pred=model.predict(X_test)
# y_test=y_test.to_numpy()
l=[np.argmax(i) for i in y_pred]
M=[np.argmax(i) for i in y_test]
display_metrics(M,l)

0.6726804123711341
0.6588014450170528
0.6738099042702682
0.6493343559060594


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
